In [5]:
from concurrent.futures import ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime


# 전체 페이지 수 가져오기
def get_page_num():
    url = 'https://www.pickuplimes.com/recipe/'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    page_nums = soup.find('div', {'class': 'pagination'})
    max_num = page_nums.find_all('span', {'class': 'page-text'})[1].text.split('of ')
    nums = list(range(1, int(max_num[1]) + 1))

    return nums

# 입력한 페이지의 전체 레시피 링크 가져오기
def get_links(i):
    link_list = list()
    url = 'https://www.pickuplimes.com/recipe/?page=' + str(i)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    lis = soup.find_all('li', {'class': 'flex-item slide-up-ani'})

    for li in lis:
        link_list.append('https://www.pickuplimes.com' + li.contents[1].get('href'))

    return link_list

# 입력한 링크의 출처(site), 레시피명(title), 재료 리스트(ingredients), 조리시간(time), 분량(serving), 레시피(recipe),
# 영양 성분(nutrition), 댓글 리스트(comments), 이미지(image) 가져오기
def get_contents(url):
    contents = dict()
    print(url)
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')

    # 출처(site)
    # 함수에 입력받는 url 사용

    # 레시피명(title)
    title = soup.find('h1').text

    # 재료 리스트(ingredients),
    ingredients_raw = soup.find('section', {'style': 'position: sticky; top: 10rem;'})
    ingredients = ingredients_raw.find_all('div', {'class': 'ingredient-container'})
    ing_list = list()
    for ingredient in ingredients:
        ing_list.append(ingredient.text.replace('\n', ''))

    # 조리시간(time)
    time = soup.find('span', {'style': 'font-weight: 400;'}).text

    # 분량(serving)
    servings_raw = soup.find('div', {'class': 'col servings-scaling'}).find('div', {'class': 'input-group'})
    servings = servings_raw.find('input').get('value')
    serving = servings + ' serving'

    # 레시피(recipe)
    recipe_raw = soup.find('div', {'class': 'directionlist'}).find('ol').find_all('li')
    recipe_list = list()
    for i in range(len(recipe_raw)):
        recipe_list.append(str(i + 1) + ". " + recipe_raw[i].text.replace('\n', ''))

    # 영양 성분(nutrition)
    nutrition = soup.find('div', {'id': 'nut-info'})

    calories = nutrition.find('p').text.split(' ')[1] + ' kcal'
    carbs = nutrition.find('div', {'class': 'row'}).find_all('td')[13].text
    protein = nutrition.find('div', {'class': 'row'}).find_all('td')[21].text
    total_fat = nutrition.find('div', {'class': 'row'}).find_all('td')[1].text

    # 댓글 리스트(comments)
    # comments = soup.select('div[class=comment] > p > p')
    # com_list = list()
    # for comment in comments:
    #     com_list.append(comment.text)

    # 이미지(image)
    image = soup.find('div', {'class': 'col-lg-5'}).find('img', {'class': 'img-fluid'}).get('src')

    contents['site'] = url
    contents['title'] = title
    contents['ingredients'] = ing_list
    contents['time'] = time
    contents['serving'] = serving
    contents['recipe'] = recipe_list
    contents['calories'] = calories
    contents['carbs'] = carbs
    contents['protein'] = protein
    contents['total_fat'] = total_fat
    # contents['comments'] = com_list
    contents['image'] = image

    return contents

# 전체 페이지 레시피 댓글 가져오기
def get_all_page_comment(nums):
    total = dict()
    title_comments = list()
    a = 0

    with ThreadPoolExecutor(max_workers=10) as executor:
        for num in nums:
            links = get_links(num)
            for link in links:
                content = executor.submit(get_contents, link)
                title_comments.append(content.result())

                a += 1
                print(a)
    date = datetime.today().strftime("%Y%m%d")
    
    total['date'] = date
    total['pickuplimes'] = title_comments
    return total

# 메인에서 실행
if __name__ == '__main__':
    # json 저장
    nums = get_page_num()
    total = get_all_page_comment(nums)

    date = datetime.today().strftime("%Y%m%d")
        
    with open(f'D:\\fruit_hada\\crawling\\crawling_result\\{date}_pickuplimes_all.json', 'w', encoding='utf-8-sig') as file:
        json.dump(total, file, indent="\t")            
        


https://www.pickuplimes.com/recipe/creamy-peanut-butter-jackfruit-noodles-1075
1
https://www.pickuplimes.com/recipe/sweet-potato-spiced-chilli-18
2
https://www.pickuplimes.com/recipe/pickled-red-onions-388
3
https://www.pickuplimes.com/recipe/non-alcoholic-mulled-wine-1255
4
https://www.pickuplimes.com/recipe/cinnamon-spiced-orange-thyme-mocktail-1258
5
https://www.pickuplimes.com/recipe/vegan-apple-pie-704
6
https://www.pickuplimes.com/recipe/vegan-pumpkin-pie-1170
7
https://www.pickuplimes.com/recipe/homemade-pie-crust-702
8
https://www.pickuplimes.com/recipe/quick-garlic-chili-noodles-848
9
https://www.pickuplimes.com/recipe/tikka-masala-spice-mix-1131
10
https://www.pickuplimes.com/recipe/quick-tofu-tikka-masala-1097
11
https://www.pickuplimes.com/recipe/saucy-beans-on-garlic-toast-901
12
https://www.pickuplimes.com/recipe/warm-tahini-chocolate-banana-wrap-460
13
https://www.pickuplimes.com/recipe/chive-onion-whipped-tofu-toast-779
14
https://www.pickuplimes.com/recipe/teriyaki-shi

118
https://www.pickuplimes.com/recipe/blueberry-muffin-overnight-oats-467
119
https://www.pickuplimes.com/recipe/chocolate-chip-cookie-dough-overnight-oats-469
120
https://www.pickuplimes.com/recipe/apple-pie-overnight-oats-466
121
https://www.pickuplimes.com/recipe/cashew-butter-465
122
https://www.pickuplimes.com/recipe/black-forest-overnight-oats-464
123
https://www.pickuplimes.com/recipe/chocolate-coconut-peanut-butter-balls-385
124
https://www.pickuplimes.com/recipe/savoury-sun-dried-tomato-olive-muffins-316
125
https://www.pickuplimes.com/recipe/crunchy-sesame-snaps-456
126
https://www.pickuplimes.com/recipe/vanilla-caramel-cake-459
127
https://www.pickuplimes.com/recipe/raspberry-rose-chocolate-bark-404
128
https://www.pickuplimes.com/recipe/coconut-thai-green-curry-noodle-soup-377
129
https://www.pickuplimes.com/recipe/vegan-thai-green-curry-paste-376
130
https://www.pickuplimes.com/recipe/one-pot-golden-red-lentil-and-mushroom-curry-soup-375
131
https://www.pickuplimes.com/re

236
https://www.pickuplimes.com/recipe/vegetable-noodle-stir-fry-with-peanut-lime-sauce-131
237
https://www.pickuplimes.com/recipe/mushroom-chickpea-coconut-curry-130
238
https://www.pickuplimes.com/recipe/quinoa-salad-with-roasted-garlic-dressing-129
239
https://www.pickuplimes.com/recipe/apple-ginger-rhubarb-cobbler-132
240
https://www.pickuplimes.com/recipe/mexican-inspired-spice-mix-107
241
https://www.pickuplimes.com/recipe/quick-guacamole-136
242
https://www.pickuplimes.com/recipe/bold-filling-lentil-taco-salad-135
243
https://www.pickuplimes.com/recipe/mashed-chickpea-dill-sandwich-134
244
https://www.pickuplimes.com/recipe/fresh-salad-rolls-with-a-creamy-peanut-hoisin-sauce-133
245
https://www.pickuplimes.com/recipe/ranch-flavoured-roasted-chickpeas-143
246
https://www.pickuplimes.com/recipe/ranch-roasted-potato-wedges-142
247
https://www.pickuplimes.com/recipe/vegan-ranch-mayo-dip-140
248
https://www.pickuplimes.com/recipe/2-minute-chocolate-almond-spread-139
249
https://www.p

346
https://www.pickuplimes.com/recipe/roasted-lemon-turmeric-hummus-161
347
https://www.pickuplimes.com/recipe/tahini-vinaigrette-dressing-268
348
https://www.pickuplimes.com/recipe/one-bowl-oatmeal-blueberry-muffins-269
349
https://www.pickuplimes.com/recipe/classic-creamy-steel-cut-oats-270
350
https://www.pickuplimes.com/recipe/coconut-curry-roasted-squash-soup-271
351
https://www.pickuplimes.com/recipe/loaded-avocado-hummus-veggie-sandwich-272
352
https://www.pickuplimes.com/recipe/one-bowl-chocolate-banana-bread-273
353
https://www.pickuplimes.com/recipe/classic-garlic-lemon-hummus-167
354
https://www.pickuplimes.com/recipe/mixed-berry-granola-yogurt-parfait-274
355
https://www.pickuplimes.com/recipe/crunchy-pecan-cranberry-granola-122
356
https://www.pickuplimes.com/recipe/chocolate-hazelnut-milkshake-smoothie-275
357
https://www.pickuplimes.com/recipe/cold-busting-green-smoothie-276
358
https://www.pickuplimes.com/recipe/ruby-red-green-smoothie-277
359
https://www.pickuplimes.c

In [4]:
# 레시피 정보 가져오기
url = 'https://www.pickuplimes.com/recipe/creamy-peanut-butter-jackfruit-noodles-1075'
contents = dict()
print(url)
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

# 출처(site)
# 함수에 입력받는 url 사용

# 레시피명(title)
title = soup.find('h1').text

# 재료 리스트(ingredients),
ingredients_raw = soup.find('section', {'style': 'position: sticky; top: 10rem;'})
ingredients = ingredients_raw.find_all('div', {'class': 'ingredient-container'})
ing_list = list()
for ingredient in ingredients:
    ing_list.append(ingredient.text.replace('\n', ''))

# 조리시간(time)
time = soup.find('span', {'style': 'font-weight: 400;'}).text

# 분량(serving)
servings_raw = soup.find('div', {'class': 'col servings-scaling'}).find('div', {'class': 'input-group'})
servings = servings_raw.find('input').get('value')
serving = servings + ' serving'

# 레시피(recipe)
recipe_raw = soup.find('div', {'class': 'directionlist'}).find('ol').find_all('li')
recipe_list = list()
for i in range(len(recipe_raw)):
    recipe_list.append(str(i + 1) + ". " + recipe_raw[i].text.replace('\n', ''))

# 영양 성분(nutrition)
nutrition = soup.find('div', {'id': 'nut-info'})

calories = nutrition.find('p').text.split(' ')[1] + ' kcal'
carbs = nutrition.find('div', {'class': 'row'}).find_all('td')[13].text
protein = nutrition.find('div', {'class': 'row'}).find_all('td')[21].text
total_fat = nutrition.find('div', {'class': 'row'}).find_all('td')[1].text

# 댓글 리스트(comments)
# comments = soup.select('div[class=comment] > p > p')
# com_list = list()
# for comment in comments:
#     com_list.append(comment.text)

# 이미지(image)
image = soup.find('div', {'class': 'col-lg-5'}).find('img', {'class': 'img-fluid'}).get('src')

contents['site'] = url
contents['title'] = title
contents['ingredients'] = ing_list
contents['time'] = time
contents['serving'] = serving
contents['recipe'] = recipe_list
contents['calories'] = calories
contents['carbs'] = carbs
contents['protein'] = protein
contents['total_fat'] = total_fat
# contents['comments'] = com_list
contents['image'] = image
print(contents)

https://www.pickuplimes.com/recipe/creamy-peanut-butter-jackfruit-noodles-1075
{'site': 'https://www.pickuplimes.com/recipe/creamy-peanut-butter-jackfruit-noodles-1075', 'title': 'Creamy Peanut Butter Jackfruit Noodles', 'ingredients': ['1bell pepper, sliced', '2cups(356 g)canned young green jackfruit, roughly chopped', '1 Tbsp(15 mL)vegetable oil', '14 oz(400 g)pre-cooked udon noodles', '1stalk green onion, sliced, light and dark green parts separated ', '½cup(120 mL)canned light coconut milk', '⅓cup(80 mL)water', '½cup(120 mL)peanut butter', '3 Tbsp(45 mL)sodium-reduced\xa0soy sauce', '1 Tbsp(15 mL)sesame oil', '½vegetable bouillon cube, crushed', '½lime, juiced', '1clove garlic, crushed', '2 tsp(5 g)cornstarch', '2 tsp(8 g)brown sugar, lightly packed', '2 tsp(10 g)crispy chili oil', '1 tsp(2 g)freshly grated ginger (optional)', 'crispy fried onions', 'lime slice', 'toasted sesame seeds'], 'time': '15 min', 'serving': '3 serving', 'recipe': ['1. To a sauté pan on medium-high heat, co